<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
def get_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

get_time()
gen_output = 0

Run date time (IST): 2023-10-07 22:05:35


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=8ad81f2924eee63df1b3a0209a8fec0f64086e95f18390cf40595981ee95bc19
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
start1_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

def read_file(path):
 df = pd.read_csv(path)
 return df

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/'

mypfs = pd.read_csv(path_mypf + 'myPortfolioStocks.csv')
mypps = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

df_mypf = pd.merge(mypfs, mypps, on="Symbol")


def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'ADANITRANS', 'SREERAYALK']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

In [5]:
df_mypf = df_mypf[df_mypf['InPortfolio'].isnull()]
cols = ['Symbol', 'Criteria', 'Strategy', 'LatestQtr', 'Target','StarStock', 'BizCheck', 'FinCheck', 'TechCheck', 'Conviction', 'Category', 'Stock']
df_mypf = df_mypf[cols]
all_prospects = get_stock_ids(df_mypf)

In [6]:
def print_distribution():

  # star stocks
  print('Star Prospects:')
  print(df_mypf['StarStock'].value_counts())
  print('-------')

  # v40/v40n stocks
  print('V40/V40N Prospects:')
  print(df_mypf['Criteria'].value_counts())
  print('-------')

  # latest quarter
  print('Latest Qtr:')
  print(df_mypf['LatestQtr'].value_counts())
  print('-------')

  # star + v40/v40n stocks
  print('Star + V40/V40N Stocks:')
  print(df_mypf[(df_mypf['Criteria'] == 'V40') | (df_mypf['Criteria'] == 'V40N')]['StarStock'].value_counts())
  print('-------')

print_distribution()

Star Prospects:
0    70
1    66
Name: StarStock, dtype: int64
-------
V40/V40N Prospects:
V40N    28
V40     24
VR       7
AR       4
V200     1
Name: Criteria, dtype: int64
-------
Latest Qtr:
1    125
0     11
Name: LatestQtr, dtype: int64
-------
Star + V40/V40N Stocks:
1    27
0    25
Name: StarStock, dtype: int64
-------


In [7]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

from google.colab import data_table

In [8]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_features(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND', 'BANDHANBNK', 'ABBOTINDIA', 'JCHAC',
              'NESTLEIND', 'PGHH', 'EQUITASBNK', 'SBICARD', 'ASTRAZEN', 'ICICIGI', 'RELAXO']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:

    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
    current_roce = float(test_list[test_list.index('ROCE')+1:test_list.index('ROCE')+2][0])
    current_roe = float(test_list[test_list.index('ROE')+1:test_list.index('ROE')+2][0])
  except ValueError as ve:
    current_pe = 1000
    current_roce = 1000
    current_roe = 1000

  return current_pe, current_roce, current_roe

In [9]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      mid_window = 50
      long_window = 200
      moving_avg = 'SMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='5y', interval='1wk')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['30_WMA'] = round(stock_df['Close'].rolling(window = 30, min_periods = 1).mean(),0)
      stock_df['Dev%_30'] = round((stock_df['Close'] - stock_df['30_WMA'])*100/stock_df['30_WMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),0)
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
      stock_df['Min'] = round(min(stock_df['Close']),0)
      stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
      #stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      #stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in all_prospects:
      #print(stock_id)
      current_pe, current_roce, current_roe = get_current_features(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      tmp['Curr_PE'] = current_pe
      tmp['Curr_ROCE%'] = current_roce
      tmp['Curr_ROE%'] = current_roe
      df_prec_dev = df_prec_dev.append(tmp)
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()

In [23]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-5:]

,Close,30_WMA,Dev%_30,Stock,Max,Min,Gained%
Date,,,,,,,
2023-09-04 00:00:00+05:30,19820.0,18496.0,7.16,^NSEI,20192.0,16945.0,16.97
2023-09-11 00:00:00+05:30,20192.0,18570.0,8.74,^NSEI,20192.0,16945.0,19.16
2023-09-18 00:00:00+05:30,19674.0,18644.0,5.53,^NSEI,20192.0,16945.0,16.11
2023-09-25 00:00:00+05:30,19638.0,18712.0,4.95,^NSEI,20192.0,16945.0,15.89
2023-10-02 00:00:00+05:30,19654.0,18787.0,4.61,^NSEI,20192.0,16945.0,15.98


In [11]:
stock_ids = df_common_features['Stock'].values
df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)

In [12]:
df_prospects = pd.merge(df_common_features, df_mypf, on='Stock')

In [13]:
#df_common_features['Discount%PE'] = round((df_common_features['MedianPE']-df_common_features['Curr_PE'])*100/(df_common_features['MedianPE']),0)
#df_prospects['Target'] = np.where(df_prospects['Strategy']== 'BTT', df_prospects["Target"], df_prospects['Max'])
df_prospects['Upside%Target'] = round((df_prospects['Max']-df_prospects['Close'])*100/(df_prospects['Close']),0)
df_prospects.drop([ '30_WMA', 'Symbol', 'Target', 'Max', 'Close'], axis=1, inplace=True)
df_prospects = df_prospects.set_index('Stock')

In [14]:
# top 10 prospects to invest from BTT stocks
df_prospects[df_prospects['Strategy'] == 'BTT'].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_30,Min,Gained%,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,
PNB.NS,34.65,35.0,119.29,19.2,4.1,3.34,VR,BTT,1,1,1.0,1.0,1.0,M,BANKS,19.0
PNB.NS,34.65,35.0,119.29,19.2,4.1,3.34,VR,BTT,1,1,1.0,1.0,1.0,M,BANKS,19.0
MOTILALOFS.NS,26.15,556.0,66.30,9.6,12.9,15.60,V40N,BTT,1,1,0.0,NaN,0.0,L,FINANCE,18.0
AKZOINDIA.NS,0.56,1993.0,25.83,31.0,34.7,26.10,V40,BTT,1,1,1.0,1.0,NaN,M,NaN,13.0
ASIANPAINT.NS,1.68,2705.0,18.14,65.8,34.4,27.70,V40,BTT,1,1,1.0,1.0,1.0,H,PAINTS,10.0
BAJAJHIND.NS,42.78,10.0,157.00,1000.0,1000.0,1000.00,VR,BTT,1,0,1.0,1.0,1.0,M,SUGAR,8.0
BAJAJHIND.NS,42.78,10.0,157.00,1000.0,1000.0,1000.00,VR,BTT,1,0,1.0,1.0,1.0,M,SUGAR,8.0
ABBOTINDIA.NS,2.49,17860.0,28.98,47.3,41.1,31.60,V40,BTT,1,1,1.0,1.0,NaN,M,PHARMA,4.0
BLUESTARCO.NS,15.40,579.0,50.67,58.3,24.4,23.40,AR,BTT,1,1,1.0,1.0,1.0,H,AC,3.0


In [15]:
# prospects to invest from star + V40/V40N/VR + below 200 DMA
df_prospects[(df_prospects['Criteria'].notnull()) & (df_prospects['StarStock'] == 1) & (df_prospects['Dev%_30'] < 5)].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_30,Min,Gained%,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,
VINATIORGA.NS,0.52,1755.0,5.97,43.4,30.40,22.6,V40N,NaN,1,1,1.0,NaN,NaN,M,NaN,25.0
DABUR.NS,-0.52,503.0,8.78,56.5,22.70,19.5,V40,NaN,1,1,0.0,1.0,1.0,H,FMCG,17.0
AKZOINDIA.NS,0.56,1993.0,25.83,31.0,34.70,26.1,V40,BTT,1,1,1.0,1.0,NaN,M,NaN,13.0
ITC.NS,1.88,320.0,38.17,27.8,39.00,29.1,V40,NaN,1,1,0.0,NaN,1.0,H,FMCG,11.0
ASIANPAINT.NS,1.68,2705.0,18.14,65.8,34.40,27.7,V40,BTT,1,1,1.0,1.0,1.0,H,PAINTS,10.0
MARICO.NS,0.65,473.0,13.63,51.2,41.90,35.8,V40,NaN,1,1,0.0,NaN,1.0,M,FMCG,9.0
ICICIBANK.NS,2.07,804.0,17.56,17.8,6.32,17.2,V40,NaN,1,1,0.0,NaN,1.0,H,BANKS,5.0
ABBOTINDIA.NS,2.49,17860.0,28.98,47.3,41.10,31.6,V40,BTT,1,1,1.0,1.0,NaN,M,PHARMA,4.0


In [16]:
# top 10 prospects to invest from star stocks
df_prospects[df_prospects['StarStock'] == 1].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_30,Min,Gained%,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,
ADANIGREEN.NS,0.13,486.0,97.79,124.0,7.81,22.80,NaN,NaN,1,1,0.0,1.0,0.0,L,NaN,200.0
MUTHOOTFIN.NS,3.18,909.0,32.35,12.7,12.10,17.90,NaN,NaN,1,1,0.0,NaN,0.0,L,FINANCE,36.0
KANSAINER.NS,9.39,251.0,29.00,48.4,14.50,10.70,V40N,NaN,1,1,1.0,NaN,NaN,M,NaN,31.0
VINATIORGA.NS,0.52,1755.0,5.97,43.4,30.40,22.60,V40N,NaN,1,1,1.0,NaN,NaN,M,NaN,25.0
BPCL.NS,-2.20,295.0,17.69,3.7,6.86,6.34,NaN,NaN,1,1,0.0,1.0,NaN,L,NaN,25.0
PNB.NS,34.65,35.0,119.29,19.2,4.10,3.34,VR,BTT,1,1,1.0,1.0,1.0,M,BANKS,19.0
PNB.NS,34.65,35.0,119.29,19.2,4.10,3.34,VR,BTT,1,1,1.0,1.0,1.0,M,BANKS,19.0
MOTILALOFS.NS,26.15,556.0,66.30,9.6,12.90,15.60,V40N,BTT,1,1,0.0,NaN,0.0,L,FINANCE,18.0
TASTYBITE.NS,29.19,8021.0,106.85,1000.0,1000.00,1000.00,NaN,NaN,1,1,1.0,NaN,NaN,M,NaN,18.0


In [17]:
# top 10 prospects to invest from  V40/V40N/VR
df_prospects[df_prospects['Criteria'].notnull()].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_30,Min,Gained%,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,
JCHAC.NS,8.06,952.0,26.45,1000.0,1000.00,1000.00,V40N,NaN,1,0,1.0,NaN,NaN,M,NaN,162.0
TANLA.NS,14.88,521.0,96.03,28.4,37.90,31.20,AR,ATH,0,0,1.0,1.0,0.0,M,IT,99.0
SFL.NS,-1.57,994.0,9.13,56.2,13.60,12.50,V40N,NaN,1,0,1.0,NaN,NaN,M,NaN,79.0
SYMPHONY.NS,-4.90,833.0,4.69,54.2,14.80,13.50,V40N,NaN,1,0,1.0,NaN,NaN,M,NaN,68.0
LALPATHLAB.NS,16.74,1808.0,37.15,78.5,18.20,15.00,V40N,NaN,1,0,1.0,NaN,NaN,M,NaN,64.0
WHIRLPOOL.NS,13.02,1257.0,31.27,99.6,8.43,6.37,V40,NaN,1,0,0.0,NaN,1.0,M,NaN,61.0
DREAMFOLKS.NS,-5.30,364.0,40.74,37.2,76.80,60.20,AR,ATH,0,0,0.0,1.0,0.0,L,MISC,52.0
PFIZER.NS,4.81,3463.0,15.19,119.0,27.20,19.80,V40,NaN,1,0,0.0,NaN,0.0,L,PHARMA,50.0
TTKPRESTIG.NS,4.92,669.0,16.37,43.7,18.10,13.80,V40N,NaN,1,0,1.0,NaN,NaN,M,NaN,45.0


In [18]:
# top 10 prospects to invest below 200 DMA
df_prospects[df_prospects['Dev%_30'] < 0].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_30,Min,Gained%,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,
ALKYLAMINE.NS,-4.93,2148.0,7.51,62.20,41.30,41.40,NaN,NaN,0,0,1.0,1.0,0.0,M,CHEMICALS,89.0
SFL.NS,-1.57,994.0,9.13,56.20,13.60,12.50,V40N,NaN,1,0,1.0,NaN,NaN,M,NaN,79.0
SYMPHONY.NS,-4.90,833.0,4.69,54.20,14.80,13.50,V40N,NaN,1,0,1.0,NaN,NaN,M,NaN,68.0
DREAMFOLKS.NS,-5.30,364.0,40.74,37.20,76.80,60.20,AR,ATH,0,0,0.0,1.0,0.0,L,MISC,52.0
UPL.NS,-6.40,578.0,5.91,16.90,14.30,13.40,NaN,NaN,1,0,0.0,NaN,NaN,L,NaN,31.0
VEDL.NS,-12.32,221.0,0.77,11.10,21.20,20.40,NaN,NaN,1,0,0.0,NaN,NaN,L,NaN,30.0
HINDPETRO.NS,-0.87,204.0,26.35,4.38,-8.08,-18.90,NaN,NaN,1,0,0.0,0.0,NaN,L,NaN,28.0
BPCL.NS,-2.20,295.0,17.69,3.70,6.86,6.34,NaN,NaN,1,1,0.0,1.0,NaN,L,NaN,25.0
HONAUT.NS,-1.14,34942.0,11.57,1000.00,1000.00,1000.00,V40N,NaN,1,0,1.0,NaN,NaN,M,NaN,25.0


In [19]:
# star + v40/v40n prospects (for breakout)

tmp_df = df_prospects[df_prospects['StarStock'] == 1]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=True)

qualified stocks: 13


,Dev%_30,Min,Gained%,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,
ANGELONE.NS,30.54,1042.0,89.79,17.8,44.00,47.1,V40N,NaN,1,1,1.0,1.0,NaN,M,FINANCE,0.0
ASTRAZEN.NS,26.94,3014.0,56.80,71.7,30.90,23.0,V40N,NaN,1,1,1.0,1.0,NaN,M,NaN,0.0
BAJFINANCE.NS,18.73,5569.0,46.68,40.1,11.80,23.5,V40,BTT,1,1,1.0,1.0,1.0,H,FINANCE,0.0
TITAN.NS,14.60,2316.0,42.91,91.3,25.10,30.8,V40,NaN,1,1,0.0,NaN,1.0,H,JEWELLERY,0.0
UJJIVANSFB.NS,54.10,23.0,161.30,1000.0,1000.00,1000.0,V40N,BTT,1,1,1.0,1.0,1.0,M,BANKS,0.0
CAPLIPOINT.NS,30.59,593.0,82.12,20.8,26.30,22.3,V40N,NaN,1,1,1.0,1.0,NaN,M,NaN,1.0
BAJAJ-AUTO.NS,12.71,3330.0,50.59,21.7,26.20,20.2,V40,NaN,1,1,0.0,1.0,NaN,L,NaN,2.0
POLYCAB.NS,32.68,2500.0,110.37,54.4,27.00,20.0,V40N,NaN,1,1,1.0,NaN,1.0,M,CABLES,2.0
COLPAL.NS,13.93,1428.0,39.53,125.0,179.00,158.0,V40,BTT,1,1,0.0,1.0,1.0,H,FMCG,3.0


In [20]:
# prospects for interactive analysis

data_table.DataTable(df_prospects, include_index=True)

,Dev%_30,Min,Gained%,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,
3MINDIA.NS,16.42,21699.0,43.56,70.80,31.40,23.40,V40N,NaN,1,1,1.0,NaN,NaN,M,NaN,6.0
ABBOTINDIA.NS,2.49,17860.0,28.98,47.30,41.10,31.60,V40,BTT,1,1,1.0,1.0,NaN,M,PHARMA,4.0
ACC.NS,9.30,1650.0,21.82,44.40,10.00,7.06,NaN,NaN,0,0,0.0,1.0,0.0,L,CEMENT,29.0
ADANIENT.NS,9.75,1314.0,88.59,102.00,9.49,9.63,NaN,NaN,1,0,0.0,1.0,1.0,M,NaN,62.0
ADANIGREEN.NS,0.13,486.0,97.79,124.00,7.81,22.80,NaN,NaN,1,1,0.0,1.0,0.0,L,NaN,200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VOLTAS.NS,8.60,737.0,20.09,88.60,9.67,4.42,NaN,NaN,1,0,0.0,NaN,0.0,L,AC,47.0
WHIRLPOOL.NS,13.02,1257.0,31.27,99.60,8.43,6.37,V40,NaN,1,0,0.0,NaN,1.0,M,NaN,61.0
WSTCSTPAPR.NS,28.98,460.0,54.22,4.63,52.90,44.80,VR,ATH,0,0,0.0,NaN,NaN,L,PAPER,1.0


In [21]:
# save the output
if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_prec_dev = df_prec_dev.reset_index()
  df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/myProspects-Upside-Analysis-ATH.csv', index=False)

In [22]:
# stock in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BERGEPAINT.NS')[-21::5]

,Close,30_WMA,Dev%_30,Stock,Max,Min,Gained%
Date,,,,,,,
2023-05-15 00:00:00+05:30,518.0,487.0,6.40,BERGEPAINT.NS,701.0,447.0,15.92
2023-06-19 00:00:00+05:30,560.0,495.0,13.12,BERGEPAINT.NS,701.0,447.0,25.27
2023-07-24 00:00:00+05:30,563.0,507.0,11.10,BERGEPAINT.NS,701.0,447.0,26.01
2023-08-28 00:00:00+05:30,600.0,528.0,13.61,BERGEPAINT.NS,701.0,447.0,34.20
2023-10-02 00:00:00+05:30,565.0,548.0,3.17,BERGEPAINT.NS,701.0,447.0,26.48
